# Procesamiento de climatologías

Este _notebook_ contiene programas en Python para el procesamiento de climatologías. Una climatología se define como un conjunto de medidas estadísticas que resumen series temporales de variables climáticas como precipitación, temperatura, radiación solar y velocidad del viento, entre otras.

**Bibliotecas**

In [ ]:
import os

import rasterio

from shapely.geometry import Point

import pandas as pd
import geopandas as gpd

## Entrada

In [ ]:
# Directorio de datos de entrada
INPUT_DATA_DIR = "C:/Users/mfvargas/Desktop/procesamiento-climatologias/datos/chirps"

## Procesamiento

In [ ]:
# Recorrido de los archivos del directorio de entrada
for file in os.listdir(INPUT_DATA_DIR):
    # DataFrame
    centros_gdf = gpd.GeoDataFrame(columns=['geometry', 'x', 'y'], crs={"init": "epsg:4326"})
    
    dataset = rasterio.open(INPUT_DATA_DIR + "/" + file)
    print("Archivo:", file, "Ancho:", dataset.width, "Largo:", dataset.height)
    
    # Recuperación de los valores de la banda 1 en un arreglo
    band = dataset.read(1)
    
    # Recorrido de las filas y columnas de la banda
    i = 0
    t = 0
    for row in band:
        j = 0
        for value in row:
            coordinates = dataset.transform * (j, i)
            x = coordinates[0]
            y = coordinates[1]
            point = Point(coordinates)
            print(t, i, j, x, y, point, value)
            centros_gdf.loc[t] = [point] + [x] + [y]
            j += 1
            t += 1
        i += 1

In [ ]:
centros_gdf

In [ ]:
centros_gdf.to_file(INPUT_DATA_DIR + "/" + "centros.shp")